In [1]:
#Pre-process the Reviews
import random
import sys
import random
import re
from collections import defaultdict

ifile1 = open("full-meta-data.txt")
revid = 0
users = defaultdict(list)
items = defaultdict(list)
spam = defaultdict(list)
spammer = defaultdict(list)
revindex = {}
spamindex = {}
hotelindex = {}
spam_1 = defaultdict(list)
nspam_1 = defaultdict(list)
for ln in ifile1:
    parts = ln.strip().split("\t")
    #print(parts[0])
    users[parts[0]].append(revid)
    items[parts[1]].append(revid)
    if parts[3]=="-1":
        spam[parts[1]].append(revid)
        spammer[parts[0]].append(revid)
        spam_1[parts[1]].append(parts[0])
    else:
        nspam_1[parts[1]].append(parts[0])
    spamindex[revid] = int(parts[3])
    revindex[revid] = parts[0]
    hotelindex[revid] = parts[1]
    revid = revid + 1
ifile1.close()




In [ ]:
#Read the reduced review dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import sys
import re
ifile = open("filteredreviews.txt")
H1 = defaultdict(list)
U1 = defaultdict(list)
for ln in ifile:
    parts = ln.strip().split(":")
    if len(parts)<2:
        continue
    parts1 = parts[1].split()
    if len(parts1) < 4:
        continue
    for p in parts1:
        H1[parts[0]].append(int(p))
ifile.close()

#print(H1)

#sys.exit()
ifile = open("reviewContent.txt",encoding='utf8')
ifile1 = open("full-meta-data.txt")
s_words = []
sfile = open("Words.txt")
for ln in sfile:
    s_words.append(ln.strip())
sfile.close()
stopwords = []
sfile = open("stopwords.txt")
for ln in sfile:
    stopwords.append(ln.strip())
sfile.close()

flags = (re.UNICODE if sys.version < '3' and type(text) is unicode
         else 0)
ofile = open("all_revs_opspm.txt",'w')
ofile1 = open("metadata_opspm.txt",'w')
cnt = 0
revid = 0
windex = defaultdict(list)
#Tunable parameter to keep non-sentiment words
PNonSentWords = 0.25
revid = 0
WORDS = defaultdict(list)
for ln in ifile:
    ln1 = ifile1.readline()
    parts1 = ln1.strip().split("\t")
    if revid not in H1[parts1[1]]:
        revid = revid + 1
        continue
    keep = []
    parts = ln.strip().split("\t")
    for word in re.findall(r"\w[\w']*", parts[3], flags=flags):
        if word.isdigit() or len(word)==1:
            continue
        word_lower = word.lower()
        if word_lower in stopwords:
            continue
        if word_lower in s_words:
            keep.append(word_lower)
        #elif random.random() < PNonSentWords:
        #    keep.append(word_lower)
    if len(keep)>5:
        #words_filtered.append(" ".join(keep))
        WORDS[revid] = keep
    revid = revid + 1
ofile.close()
ofile1.close()
ifile.close()
ifile1.close()

print(WORDS)


In [ ]:
#SVM for spam/non-spam prediction
from sklearn import svm
#from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (precision_score,recall_score,f1_score)
from sklearn.feature_extraction.text import CountVectorizer

import sys

words = []
Y= []
for w in WORDS.keys():
    if len(WORDS[w])==0:
        continue
    words.append(" ".join(WORDS[w]))
    if spamindex[w]==-1:
        Y.append(1)
    else:
        Y.append(0)
#print(words)
#sys.exit()
tf_transformer = CountVectorizer()
f = tf_transformer.fit_transform(words)
clf = svm.LinearSVC(C=1)
clf.fit(f,Y)
p = clf.predict(f)
print(f1_score(Y,p))

rangelower = 0
rangehigher = 1


C = clf.coef_
scaler = MinMaxScaler(feature_range=(rangelower,rangehigher))
vals = []
for i, j in zip(*C.nonzero()):
    vals.append([C[i,j]])
scaler.fit(vals)
V1 = scaler.transform(vals)
rows,cols = C.nonzero()
#print(cols)

r_wts = {}
idx = 0
for i, j in zip(*C.nonzero()):
    for k in tf_transformer.vocabulary_.keys():
        if tf_transformer.vocabulary_[k]==j:
            r_wts[k] = V1[idx][0]
            break
    idx = idx + 1
print(r_wts)

In [ ]:
#SVM for ratings (low/high) predictions

ifile1 = open("full-meta-data.txt")
ratings = {}
idx = 0
for ln in ifile1:
    ratings[idx] = int(float(ln.strip().split("\t")[2]))
    idx = idx + 1
ifile1.close()

RIndex1 = defaultdict(dict)
for h in H1:
    for r in H1[h]:
        if spamindex[r]==-1:
            if ratings[r] >=4:
                RIndex1[h][r] = -1
            elif ratings[r] <=2:
                RIndex1[h][r] = -2
        else:
            if ratings[r] >=4:
                RIndex1[h][r] = 1
            elif ratings[r] <=2:
                RIndex1[h][r] = 2

words = []
Y= []

for r in RIndex1:
    for e1 in RIndex1[r]:
        if len(WORDS[e1])==0:
            continue
        words.append(" ".join(WORDS[e1]))
        if RIndex1[r][e1] == -1:
            RIndex1[r][e1] = -2
        if RIndex1[r][e1]==1 or RIndex1[r][e1]==-1:
            Y.append(1)
        else:
            Y.append(0)
tf_transformer = CountVectorizer()
f = tf_transformer.fit_transform(words)
clf = svm.LinearSVC(C=1)
clf.fit(f,Y)
p = clf.predict(f)
print(f1_score(Y,p))

rangelower = 0
rangehigher = 1


C = clf.coef_
scaler = MinMaxScaler(feature_range=(rangelower,rangehigher))
vals = []
for i, j in zip(*C.nonzero()):
    vals.append([C[i,j]])
scaler.fit(vals)
V1 = scaler.transform(vals)
rows,cols = C.nonzero()

r_wts_1 = {}
idx = 0
for i, j in zip(*C.nonzero()):
    for k in tf_transformer.vocabulary_.keys():
        if tf_transformer.vocabulary_[k]==j:
            r_wts_1[k] = V1[idx][0]
            break
    idx = idx + 1

In [ ]:
#SAMPLER
import operator
import sys
cwt = 0.01
Sample = {}
Sample_r = {}
margs = {}
margs_r = {}
iters = {}
iters_r = {}
users_s= defaultdict(list)
expns = defaultdict(dict)

#print(H1)
#sys.exit()
for h in H1:
    for i,r in enumerate(H1[h]):
        Sample[r] = random.randint(0,1)
        Sample_r[r] = random.randint(0,1)
        margs[r] = 0
        margs_r[r] = 0
        iters[r] = 0
        iters_r[r] = 0
        users_s[revindex[r]].append(r)
#print(users_s)
#sys.exit()
exp = 0
VR = 0.9
VR1 = 0.9
for t in range(0,500000,1):
    #for h in H1:
        #for i,r in enumerate(H1[h]):
    h = random.choice(list(H1.keys()))
    if len(H1[h])==0:
        continue
    ix = random.randint(0,len(H1[h])-1)
    r = H1[h][ix]
   # print(r)
    if len(WORDS[r])==0 or ratings[r]==3:
        continue
    if random.random() < 0.5:
        P = 0
        N = 0
        S1 = 0
        S2 = 0
        mx = 0
        exp1 = 0
        for w in WORDS[r]:
            if w in r_wts:
                P = P + r_wts[w]
                N = N + (1-r_wts[w])
                if r_wts[w] > mx:
                    mx = P
                if r not in expns or w not in expns[r]:
                    expns[r][w] = r_wts[w]
                else:
                    expns[r][w] = expns[r][w] + r_wts[w]
       # print(N)
        for r1 in users_s[revindex[r]]:
            if r1==r:
                continue
            if r1 not in Sample:
                continue
            if Sample[r1]==1:
                P = P + VR1
                if r not in expns or "SUSER" not in expns[r]:
                    expns[r]['SUSER'] = VR1
                else:
                    expns[r]['SUSER'] = expns[r]['SUSER'] + VR1 
            else:
                N = N + VR1
        for r1 in H1[h]:
            if r1==r:
                continue
            if Sample_r[r]!=Sample_r[r1]:
                if Sample[r1]==0:
                    P = P + VR
                    exp1 = exp1 + 0.1
                    if r not in expns or "JNTM" not in expns[r]:
                        expns[r]['JNTM'] = VR
                    else:
                        expns[r]['JNTM'] = expns[r]['JNTM'] + VR 
                else:
                    N = N + VR
        if exp1 > mx:
            exp = exp+1
        if P >= N-0.75:
            Sample[r] = 1
            margs[r] = margs[r] + 1
        else:
            Sample[r] = 0  
        iters[r] = iters[r] + 1
    else:
        P1 = 0
        N1 = 0
        for w in WORDS[r]:
            if w in r_wts_1:
                P1 = P1 + r_wts_1[w]
                N1 = N1 + (1-r_wts_1[w])            
        for r1 in H1[h]:
            if r1==r:
                continue
            if Sample[r]!=Sample[r1]:
                if Sample_r[r1]==0:
                    P1 = P1 + 0.1
                else:
                    N1 = N1 + 0.1      
        if P1 >= N1+0.75:
            Sample_r[r] = 1
            margs_r[r] = margs_r[r] + 1
        else:
            Sample_r[r] = 0
        iters_r[r] = iters_r[r] + 1
    
        
for m in margs:
    if iters[m] > 0:
        margs[m] = margs[m]/iters[m]
    if iters_r[m] > 0:
        margs_r[m] = margs_r[m]/iters_r[m]
preds = []
true = []
for m in margs.keys():
    if len(WORDS[m])==0 or ratings[m]==3:
        continue
    if margs[m] > 0.5:
        preds.append(1)
    else:
        preds.append(0)
    if spamindex[m] ==-1:
        true.append(1)
    else:
        true.append(0)
print(precision_score(true,preds))
print(recall_score(true,preds))
print(f1_score(true,preds))

preds = []
true = []
preds_r = {}
for h in H1:
    for i,r in enumerate(H1[h]):
        if len(WORDS[r])==0 or ratings[r]==3:
            continue
        if r not in RIndex1[h]:
            continue
        if margs_r[r] > 0.5:
            preds.append(1)
        else:
            preds.append(0)
        if RIndex1[h][r]==1 or RIndex1[h][r]==-1:
            true.append(1)
        else:
            true.append(0)
        if preds[len(preds)-1]==true[len(true)-1]:
            preds_r[r] = preds[len(preds)-1]
  
print(precision_score(true,preds))
print(recall_score(true,preds))
print(f1_score(true,preds))




In [ ]:
#Explanations
correct_predictions = {}
for m in margs.keys():
    if len(WORDS[m])==0 or ratings[m]==3:
        continue
    if margs[m] > 0.5 and spamindex[m] ==-1:
        correct_predictions[m] = 1
fft=open("expn_n1.txt","w")     
for e in expns:
    if e in correct_predictions:
        sorted_expns = sorted(expns[e].items(), key=operator.itemgetter(1),reverse=True)
        z = 0
        for s in sorted_expns[:5]:
            z = z + s[1]
        rex = {}
        keys = []
        for s in sorted_expns[:5]:
            rex[s[0]] = s[1]/z
            keys.append(s[0])
        if "JNTM" not in keys or "SUSER" not in keys:
            continue
        sorted1 = sorted(rex.items(), key=operator.itemgetter(1),reverse=True)
        #if sorted1[0][0]=="JNTM":
        print(str(e) +" "+str(sorted1))
        gg=str(e) +":"+str(sorted1)
        fft.write(str(gg)+"\n")